In [2]:
import csv
import requests
import pandas as pd
from config import api_key
import pymongo

In [3]:
csv_file = "datasets_1919_3310_flavors_of_cacao.csv"
chocolate_df = pd.read_csv(csv_file)
chocolate_df

,Company \n(Maker-if known),Specific Bean Origin\nor Bar Name,REF,Review\nDate,Cocoa\nPercent,Company\nLocation,Rating,Bean\nType,Broad Bean\nOrigin
0,A. Morin,Agua Grande,1876,2016,63%,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676,2015,70%,France,2.75,,Togo
2,A. Morin,Atsane,1676,2015,70%,France,3.00,,Togo
3,A. Morin,Akata,1680,2015,70%,France,3.50,,Togo
4,A. Morin,Quilla,1704,2015,70%,France,3.50,,Peru
...,...,...,...,...,...,...,...,...,...
1790,Zotter,Peru,647,2011,70%,Austria,3.75,,Peru
1791,Zotter,Congo,749,2011,65%,Austria,3.00,Forastero,Congo
1792,Zotter,Kerala State,749,2011,65%,Austria,3.50,Forastero,India
1793,Zotter,Kerala State,781,2011,62%,Austria,3.25,,India


In [4]:
chocolate_columns = ["Specific Bean Origin\nor Bar Name", "Review\nDate", "Cocoa\nPercent",\
                    "Company\nLocation", "Rating", "Bean\nType", "Broad Bean\nOrigin"]
chocolate_rating = chocolate_df[chocolate_columns].copy()
chocolate_rating

,Specific Bean Origin\nor Bar Name,Review\nDate,Cocoa\nPercent,Company\nLocation,Rating,Bean\nType,Broad Bean\nOrigin
0,Agua Grande,2016,63%,France,3.75,,Sao Tome
1,Kpime,2015,70%,France,2.75,,Togo
2,Atsane,2015,70%,France,3.00,,Togo
3,Akata,2015,70%,France,3.50,,Togo
4,Quilla,2015,70%,France,3.50,,Peru
...,...,...,...,...,...,...,...
1790,Peru,2011,70%,Austria,3.75,,Peru
1791,Congo,2011,65%,Austria,3.00,Forastero,Congo
1792,Kerala State,2011,65%,Austria,3.50,Forastero,India
1793,Kerala State,2011,62%,Austria,3.25,,India


In [5]:
chocolate_rating = chocolate_rating.rename(columns={"Specific Bean Origin\nor Bar Name": "Geo_Region_for_Bean",\
                                                    "Review\nDate": "Review_Year", "Cocoa\nPercent": "Darkness",\
                                                    "Company\nLocation": "Manufacturing_Country", "Bean\nType": "Bean_Type",\
                                                    "Broad Bean\nOrigin": "Original_Country_for_Bean"})                                                    
chocolate_rating

,Geo_Region_for_Bean,Review_Year,Darkness,Manufacturing_Country,Rating,Bean_Type,Original_Country_for_Bean
0,Agua Grande,2016,63%,France,3.75,,Sao Tome
1,Kpime,2015,70%,France,2.75,,Togo
2,Atsane,2015,70%,France,3.00,,Togo
3,Akata,2015,70%,France,3.50,,Togo
4,Quilla,2015,70%,France,3.50,,Peru
...,...,...,...,...,...,...,...
1790,Peru,2011,70%,Austria,3.75,,Peru
1791,Congo,2011,65%,Austria,3.00,Forastero,Congo
1792,Kerala State,2011,65%,Austria,3.50,Forastero,India
1793,Kerala State,2011,62%,Austria,3.25,,India


In [6]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [7]:
db = client['weather_and_chocolate']
collection = db['chocolate_rating']
chocolate_rating.reset_index(inplace=False)
chocolate_rating_dict = chocolate_rating.to_dict("records")

In [8]:
collection.insert_many(chocolate_rating_dict)

In [9]:
chocolate_rating.drop_duplicates("Geo_Region_for_Bean")
chocolate_rating

,Geo_Region_for_Bean,Review_Year,Darkness,Manufacturing_Country,Rating,Bean_Type,Original_Country_for_Bean
0,Agua Grande,2016,63%,France,3.75,,Sao Tome
1,Kpime,2015,70%,France,2.75,,Togo
2,Atsane,2015,70%,France,3.00,,Togo
3,Akata,2015,70%,France,3.50,,Togo
4,Quilla,2015,70%,France,3.50,,Peru
...,...,...,...,...,...,...,...
1790,Peru,2011,70%,Austria,3.75,,Peru
1791,Congo,2011,65%,Austria,3.00,Forastero,Congo
1792,Kerala State,2011,65%,Austria,3.50,Forastero,India
1793,Kerala State,2011,62%,Austria,3.25,,India


In [10]:
regions = chocolate_rating.Geo_Region_for_Bean
for region in regions:
    print(region)

Agua Grande
Kpime
Atsane
Akata
Quilla
Carenero
Cuba
Sur del Lago
Puerto Cabello
Pablino
Panama
Madagascar
Brazil
Equateur
Colombie
Birmanie
Papua New Guinea
Chuao
Piura
Chanchamayo Province
Chanchamayo Province
Bolivia
Peru
Chulucanas, El Platanal
Tumbes, Norandino
Vanua Levu
Vanua Levu, Toto-A
Vanua Levu
Vanua Levu, Ami-Ami-CA
Los Rios, Quevedo, Arriba
Los Rios, Quevedo, Arriba
Tabasco
Bali (west), Sukrama Family, Melaya area
Madagascar, Ambolikapiky P.
Monte Alegre, D. Badero
Trinite
Vietnam
Madagascar
Chuao
Piura, Perou
Winak Coop, Napo
La Dalia, Matagalpa
Tien Giang
Makwale Village, Kyela
Momotombo
Acopagro
CIAAB Coop
Villa Andina
Gruppo Salinas
Sur del Lago
Conacado
Bolivia
Bolivia
Peru
Morobe
Dos Rios
Guayas
Chuao
Montanya
Bali, Jembrana
Madagascar
Cuyagua
Ocumare
Ghana
Ecuador
Ecuador
Ghana
LamasdelChanka, San Martin, Oro Verde coop
Bellavista Gran Pajeten, San Martin
Belize
Madagascar
Dominican Republic
Papua New Guinea
Venezuela
Peru
Piura, Blanco de Criollo
Porcelana
Nine
Chu

Bolivian
Lumas, 2015 Harvest, Batch 6, brown sugar
Lumas, 2015 Harvest, Batch 7
Tumbes, 2013 Harvest, Batch 8
Belize, 2014 Harvest, Batch 9
Cuana, 2013
Tobago
Acarigua, w/ nibs
Loma Sotavento
Grenada
Akosombo
Porcelana, Pariguan
Kuruba
Orinoco
Cuana, 2008
Peruvian Amazon
Vinces
Madagascar, Batch 59/100
Vietnam, Batch 50/100
Cuba, Batch 59/100
Peru, Batch 51/100
Ivory Coast, Batch 56/100
Sao Tome, Batch 151
Bahia, Batch 148
Lam Dong, Batch 153
Cuba
Trinidad-Tobago
Catongo
Tanzania
Rugoso
Carenero Superior
Rio Caribe
Blend
La Masica, Batch 7, FHIA
Maya Mtn, Batch 18, Heirloom
Liberia
Beniamo
San Juan Estate, Gran Couva
San Andres
La Red, Guaconejo
Kokoa Kamili
Maranura
Sambirano
Madagascar
Perfect Illusion
Bolivia, Wild Thing
La Selva
Purple Haze
The Other One, Grand Cru
Wild Thing
Excellence (US Version)
Puerto Rico
Haleiwa, O'ahu; Lonohana E., Kanahiku
Haleiwa E, O'ahu, 2014
Haleiwa E, O'ahu, 2014
Opaeula Estate, O'ahu, Nene, CG Exclusive
Opaeula Estate, O'ahu, Ele'ele
Hawaiian Crown, 

In [11]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"
query_url = f"{url}appid={api_key}&units={units}&q="
Region = []
Lat = []
Long = []
MaxTemp = []
Humidity = []
Cloudiness = []
for region in range(len(regions)):
    try:
        response = requests.get(query_url + regions[region]).json()
        Region.append(response['name'])
        Lat.append(response['coord']['lat'])
        Long.append(response['coord']['lon'])
        MaxTemp.append(response['main']['temp_max'])
        Humidity.append(response['main']['humidity'])
        Cloudiness.append(response['clouds']['all'])
    except Exception:
        print("Region not found")

Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not fou

Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not found
Region not fou

In [14]:
Data_list = []
for i in range(len(Region)):
    temp_dict = {
        "Geo_Region_for_Bean": Region[i],
        "Latitude": Lat[i],
        "Longtitude": Long[i],
        "Max Temperature": MaxTemp[i],
        "Humidity": Humidity[i],
        "Cloudiness": Cloudiness[i]
    }
    Data_list.append(temp_dict)
Data_list

[{'Geo_Region_for_Bean': 'Akata',
  'Latitude': 39.42,
  'Longtitude': 140.12,
  'Max Temperature': 24,
  'Humidity': 60,
  'Cloudiness': 75},
 {'Geo_Region_for_Bean': 'Carenero',
  'Latitude': 21.08,
  'Longtitude': -76.03,
  'Max Temperature': 25,
  'Humidity': 94,
  'Cloudiness': 95},
 {'Geo_Region_for_Bean': 'Cuba',
  'Latitude': 22,
  'Longtitude': -79.5,
  'Max Temperature': 23.37,
  'Humidity': 93,
  'Cloudiness': 94},
 {'Geo_Region_for_Bean': 'Puerto Cabello',
  'Latitude': 10.47,
  'Longtitude': -68.01,
  'Max Temperature': 26.44,
  'Humidity': 81,
  'Cloudiness': 100},
 {'Geo_Region_for_Bean': 'Panama',
  'Latitude': 9,
  'Longtitude': -80,
  'Max Temperature': 25.56,
  'Humidity': 94,
  'Cloudiness': 98},
 {'Geo_Region_for_Bean': 'Madagascar',
  'Latitude': -20,
  'Longtitude': 47,
  'Max Temperature': 9.43,
  'Humidity': 84,
  'Cloudiness': 31},
 {'Geo_Region_for_Bean': 'Brazil',
  'Latitude': 39.52,
  'Longtitude': -87.12,
  'Max Temperature': 18,
  'Humidity': 82,
  'Clou

In [15]:
Weather_df = pd.DataFrame(Data_list)
Weather_df

,Geo_Region_for_Bean,Latitude,Longtitude,Max Temperature,Humidity,Cloudiness
0,Akata,39.42,140.12,24.00,60,75
1,Carenero,21.08,-76.03,25.00,94,95
2,Cuba,22.00,-79.50,23.37,93,94
3,Puerto Cabello,10.47,-68.01,26.44,81,100
4,Panama,9.00,-80.00,25.56,94,98
...,...,...,...,...,...,...
934,Santo Domingo,18.50,-69.99,27.78,88,75
935,Republic of the Congo,-1.00,15.00,22.85,90,100
936,Peru,40.75,-86.07,18.89,77,1
937,Republic of the Congo,-1.00,15.00,22.85,90,100


In [16]:
db = client['weather_and_chocolate']
collection = db['weather']
Weather_df.reset_index(inplace=True)
Weather_dict = Weather_df.to_dict("records")

In [17]:
collection.insert_many(Weather_dict)

In [18]:
Weather_df.to_csv('Weather.csv', header=True, index=False)

In [ ]:
db.getCollection('weather').aggregate([{
$lookup:
        {from: "chocolate_rating",
        localField: "Geo_Region_for_Bean",
        foreignField : "Geo_Region_for_Bean",
        as: "weather_chocolate"}
}])